In [2]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json
from IPython.display import Image
from IPython.display import display

api_key = 'ab4a7f41157e4592aa7cf47de4be49a1'
host = 'westcentralus.api.cognitive.microsoft.com'
personGroupId = 'jiangsir_groupid2'



def create_personGroup(personGroupId, groupname, groupdata):
    print("建立一個 personGroupid = "+personGroupId)
    headers = {
        # Request headers.
        'Content-Type': 'application/json',

        # NOTE: Replace the "Ocp-Apim-Subscription-Key" value with a valid subscription key.
        'Ocp-Apim-Subscription-Key': api_key,
    }

    # Replace 'examplegroupid' with an ID you haven't used for creating a group before.
    # The valid characters for the ID include numbers, English letters in lower case, '-' and '_'. 
    # The maximum length of the ID is 64.
    #personGroupId = 'examplegroupid'
    #personGroupId = 'jiangsir_groupid2'

    # The userData field is optional. The size limit for it is 16KB.
    body = "{ 'name':'"+groupname+"', 'userData':'"+groupdata+"' }"

    try:

        # NOTE: You must use the same location in your REST call as you used to obtain your subscription keys.
        #   For example, if you obtained your subscription keys from westus, replace "westcentralus" in the 
        #   URL below with "westus".
        conn = http.client.HTTPSConnection(host)
        conn.request("PUT", "/face/v1.0/persongroups/%s" % personGroupId, body, headers)
        response = conn.getresponse()

        # 'OK' indicates success. 'Conflict' means a group with this ID already exists.
        # If you get 'Conflict', change the value of personGroupId above and try again.
        # If you get 'Access Denied', verify the validity of the subscription key above and try again.
        print(response.reason)

        conn.close()
        return personGroupId
    except Exception as e:
        print(e.args)
        
        
        
def create_a_person(personGroupId, name, descript):
    print("在 personGroupid="+personGroupId+" 裡 建立一個 person name="+name)
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
            'personGroupId':personGroupId
    })

    requestbody = '{"name":"'+name+'","userData":"'+descript+'"}'

    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("POST", "/face/v1.0/persongroups/"+personGroupId+"/persons?%s" % params, requestbody, headers)
        response = conn.getresponse()
        data = response.read()
        #print(data)
        create_a_person_json = json.loads(str(data,'UTF-8'))

        conn.close()
        return create_a_person_json['personId']
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))    
        
def add_a_person_face(imagepath, personId, personGroupId):
    print("用一個圖片放入一個 person 當中 personId="+personId)
    display(Image(url=imagepath))

    headers = {
        # Request headers
#        'Content-Type': 'application/json',
        'Content-Type': 'application/octet-stream', #上傳圖檔
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'personGroupId': personGroupId,
        #'personId': '03cb1134-ad35-4b80-8bf2-3200f44eef31',
        'personId': personId,
        #'userData': '{string}',
        #'targetFace': '{string}',
    })
#"https://lh3.googleusercontent.com/AuJtzSdWCTZ6pWW9pMxec86gVZEjP00O7qvl8RNbzYfmJvsiUfDL-BXfel5Sw2jgPNUy7rcIVQ-HFDxDEFuIZxp56NpKwOjYncgMjL_dt0-FnoBIYyUpplx4LlE5ShN2hJ3-URLwOA4=w597-h796-no"

#    requestbody = '{"url": "'+imageurl+'"}'
    requestbody = open(imagepath, "rb").read()
    

    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("POST", "/face/v1.0/persongroups/"+personGroupId+"/persons/"+personId+"/persistedFaces?%s" % params, requestbody, headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

def train_personGroup(personGroupId):
    print("開始訓練一個 personGroup personGroupId="+personGroupId+"。")
    
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
            'personGroupId':personGroupId
    })

    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("POST", "/face/v1.0/persongroups/"+personGroupId+"/train?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))    
     
    
def personGroup_status(personGroupId):
    print("personGroup_status: 查看一個 personGroup 的狀態，也就是看看訓練是否成功！")
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
        'personGroupId':personGroupId
    })

    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("GET", "/face/v1.0/persongroups/"+personGroupId+"/training?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))    


##################################################################################################
### main
##################################################################################################

trainimages = []
trainimages.append("train_face1.jpg")
trainimages.append("train_face2.jpg")
trainimages.append("train_face3.jpg")
trainimages.append("train_face4.jpg")

#trainimages.append("https://lh3.googleusercontent.com/AuJtzSdWCTZ6pWW9pMxec86gVZEjP00O7qvl8RNbzYfmJvsiUfDL-BXfel5Sw2jgPNUy7rcIVQ-HFDxDEFuIZxp56NpKwOjYncgMjL_dt0-FnoBIYyUpplx4LlE5ShN2hJ3-URLwOA4=w597-h796-no")
#trainimages.append("https://lh3.googleusercontent.com/FywffrCKSfzhFoGenppzlH_GXPeGCOT9HvizxLXOziZ2FSAvaTgFwevxanp3hv7Zntt1o80PI84PtsZC7zXvwFfK3o-boWuEPlsjmRoUCqT3ll-WdN3h7mq9lI2akW6VwVXsOiQmKmc=w597-h796-no")
#trainimages.append("https://lh3.googleusercontent.com/7bgf-psmqPJBg78Y9WNCqEP9cfIr3-XOmykuXYO2JSbpGTMG3nZ9RCuizoJbJJr-dJLkqFN6mGzGH6nmv8p3aT1RiTLHGoNmnpGnw7t-j5RCij23v85ZK0NJnKzjuZDI9wkxivttBgY=w597-h796-no")
#trainimages.append("https://lh3.googleusercontent.com/vXifjUB85Yy4IgnxX5_c83kXwF_x9eQHheHVUNVokpB0RZNpOgwijo5jclRbTxywlQaJzpZiDJlYtgI7zyYfBfIml5YeCpf0qwTqLGtfH1wiznF8rSY2pPGcNxEqJidLup5zSAUhRdg=w597-h796-no")

personGroupId = create_personGroup("jiangsir_groupid4", "personGroup namenamename", "jiangsirpersonGroup_userDatauserData")
personId = create_a_person(personGroupId, "jiangsir", "jiangsir_userDatauserData")

for trainimage in trainimages:
    add_a_person_face(trainimage, personId, personGroupId)


train_personGroup(personGroupId)
personGroup_status(personGroupId)


建立一個 personGroupid = jiangsir_groupid4
OK
在 personGroupid=jiangsir_groupid4 裡 建立一個 person name=jiangsir
用一個圖片放入一個 person 當中 personId=94686663-6c95-411c-bb36-bba07160e85c


b'{"persistedFaceId":"1bfc6081-cb3e-4be1-a7a8-2ef83a36088b"}'
用一個圖片放入一個 person 當中 personId=94686663-6c95-411c-bb36-bba07160e85c


b'{"persistedFaceId":"546ec637-a871-49bd-9ab8-a5e9d84644ab"}'
用一個圖片放入一個 person 當中 personId=94686663-6c95-411c-bb36-bba07160e85c


b'{"persistedFaceId":"ca15177d-613f-4178-bb53-12b73ab8b787"}'
用一個圖片放入一個 person 當中 personId=94686663-6c95-411c-bb36-bba07160e85c


b'{"persistedFaceId":"36bd65d0-14d1-4c69-bb47-16962571df7e"}'
開始訓練一個 personGroup personGroupId=jiangsir_groupid4。
b''
personGroup_status: 查看一個 personGroup 的狀態，也就是看看訓練是否成功！
b'{"status":"succeeded","createdDateTime":"11/6/2017 1:10:05 PM","lastActionDateTime":"11/6/2017 1:10:04 PM","message":null}'
